# Imports
Импортируем необходимые библиотеки

In [1]:
import os
import re
import csv
import numpy as np
import pandas as pd
# import codecs
from string import punctuation
from collections import defaultdict
# from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [2]:
from sklearn.metrics import log_loss

import lightautoml
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

from lightautoml.addons.interpretation import LimeTextExplainer, L2XTextExplainer

import transformers
transformers.logging.set_verbosity(50)

In [3]:
lightautoml.__version__

'0.3.3'

# Settings

In [4]:
PATH_DATA = './input/'
train_data_file = PATH_DATA + 'train.csv'
test_data_file = PATH_DATA + 'test_small.csv'

# Reading data 

In [5]:
df_train = pd.read_csv(train_data_file, encoding='utf-8')
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
df_test = pd.read_csv(test_data_file, encoding='utf-8')
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [7]:
df_test.shape

(2345796, 3)

# LightAutoML baseline model 

In [9]:
task = Task('binary', metric = 'logloss')

In [17]:
roles = {
    'target': 'is_duplicate',
    'drop':['qid1', 'qid2', 'id'],
    'text': ['question1', 'question2']
}

In [18]:
automl = TabularNLPAutoML(
    task=task,
    timeout=10 * 3600,
    cpu_limit=12,
    gpu_ids='0',
    general_params={
        'use_algos': [['nn']]
    },
    text_params={
        'lang': 'en',
    },
    nn_params={
        'stop_by_metric': True,
        'max_length': 128,
        'bs': 128,
        'n_epochs': 10,
        
    }

)

In [19]:
%%time

oof_pred = automl.fit_predict(df_train.head(375000), valid_data=df_train.tail(25000), roles=roles, verbose = 3)

[08:37:07] Stdout logging level is INFO3.
[08:37:07] Model language mode: en
[08:37:07] Task: binary

[08:37:07] Start automl preset with listed constraints:
[08:37:07] - time: 36000.00 seconds
[08:37:07] - CPU: 12 cores
[08:37:07] - memory: 16 GB

[08:37:07] Train data shape: (375000, 6)

[08:37:07] Layer 1 train process start. Time left 35999.51 secs


val: 100%|██████████| 196/196 [01:51<00:00,  1.75it/s]


[09:20:26] Epoch: 0, train loss: 0.36467331647872925, val loss: 0.2665068507194519, val metric: -0.2664885649332404


val: 100%|██████████| 196/196 [01:51<00:00,  1.75it/s]


[10:03:31] Epoch: 1, train loss: 0.24328890442848206, val loss: 0.24908995628356934, val metric: -0.249110053135613


...

[10:46:36] Epoch: 2, train loss: 0.19006593525409698, val loss: 0.24295036494731903, val metric: -0.2428887405103771


val: 100%|██████████| 196/196 [01:51<00:00,  1.75it/s]


[11:29:42] Epoch: 3, train loss: 0.14998386800289154, val loss: 0.24872171878814697, val metric: -0.24869339985056316


val: 100%|██████████| 196/196 [01:51<00:00,  1.75it/s]


[11:31:35] Early stopping: val loss: 0.24295036494731903, val metric: -0.2428887405103771
[11:31:41] Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
[11:31:41] Time left 25526.01 secs

[11:31:41] Layer 1 training completed.

[11:31:41] Automl preset training completed in 10474.04 seconds

[11:31:41] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN) 

CPU times: user 2h 4min 5s, sys: 50min 33s, total: 2h 54min 38s
Wall time: 2h 54min 34s


In [21]:
test_pred = automl.predict(df_test).data[:, 0]
test_pred

test: 100%|██████████| 18327/18327 [2:53:48<00:00,  1.76it/s]  


array([0.01497479, 0.3438352 , 0.00289229, ..., 0.00172728, 0.00402244,
       0.950194  ], dtype=float32)

In [22]:
submission = pd.DataFrame({'test_id': df_test['test_id'].values,
             'is_duplicate': test_pred})
submission

,test_id,is_duplicate
0,0,0.014975
1,1,0.343835
2,2,0.002892
3,3,0.002414
4,4,0.076350
...,...,...
2345791,2345791,0.001722
2345792,2345792,0.001752
2345793,2345793,0.001727
2345794,2345794,0.004022


In [23]:
submission.drop_duplicates(subset=['test_id']).shape

(2345796, 2)

In [24]:
submission.to_csv('LAMA_sub_full.csv', index = False)

In [53]:
submission.shape

(2345796, 2)

В итоге получаем результат **0.29334** на публичном лидерборде и **0.30149** на приватном лидерборде, что соответствует **938ому месту (топ 30%)**. 

Идеи для улучшения - построить модель, которая нацелена на задачу выявления дубликатов, а не просто рассматривает имеющиеся данные как 2 колонки несвязанного текста.